In [1]:
!pip install torchinfo

In [2]:
import SHELFI_FHE as m
import torch.nn as nn
from torchinfo import summary
import numpy as np
import time
import torch
from collections import OrderedDict

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Some helper functions

def tensor_to_numpy_arr(params_tensor):
    params_np = OrderedDict()
    #params_shape = OrderedDict()
    for key in params_tensor.keys():
        params_np[key] = torch.flatten(params_tensor[key]).numpy()
    return params_np

def numpy_arr_to_tensor(params_np, params_shape):
    params_tensor = OrderedDict()
    for key in params_np.keys():
        params_tensor[key] = torch.from_numpy(params_np[key])
        #needs torch.Size() to tuple
        params_tensor[key] = torch.reshape(params_tensor[key], tuple(list((params_shape[key]))))
    return params_tensor

def tensor_shape(params_tensor):
    params_shape = OrderedDict()
    for key in params_tensor.keys():
        params_shape[key] = params_tensor[key].size()
    return params_shape

In [4]:
# Logistic regression model

input_size = 100
num_classes = 1

model_lr = nn.Linear(input_size, num_classes)
summary(model_lr, (1, 100))

Layer (type:depth-idx)                   Output Shape              Param #
Linear                                   [1, 1]                    101
Total params: 101
Trainable params: 101
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

In [ ]:
#update models
model = model_lr
#update number of clients
N = 3
learner_data_layer = []
params = tensor_to_numpy_arr(model.state_dict())

for id in range(N):
    learner_data_layer.append(params)

# loading model params from files
# for id in range(3):
#     os.system(“python3 tabcnn_learner.py ” + str(id+1)+ ” ” + str(n)) 
#     with open(“models/model”+str(id+1)+“.pickle”, ‘rb’) as handle:
#         b = pickle.load(handle)
#     learner_data_layer.append(b)
#     with open(“models/tensor_model”+str(id+1)+“.pickle”, ‘rb’) as handle:
#         c = pickle.load(handle)
#     plaintext_data_layer.append(c)


weight_factors = np.full(N, 1/N).tolist()

print("Setup CryptoContext.")
FHE_helper = m.CKKS("ckks", 4096, 52, "./resources/cryptoparams/")
#FHE_helper = m.CKKS()

FHE_helper.genCryptoContextAndKeyGen()
#FHE_helper.loadCryptoParams()

#encrypting
enc_learner_layer = []

time_enc_s = time.time()
for key in learner_data_layer[0].keys():
	for id in range(N):
		enc_learner_layer.append(OrderedDict())
		enc_learner_layer[id][key] = FHE_helper.encrypt(learner_data_layer[id][key])
time_enc_e = time.time()
print("Encrytion done.\n")

time_enc = (time_enc_e - time_enc_s)/N
print("Encryption Time: {}".format(time_enc))
#print(FHE_helper.decrypt(enc_res_learner[0][“conv1.weight”], int(learner_data_layer[0][“fc1.weight”].size)))


#weighted average
eval_data = copy.deepcopy(learner_data_layer[0])

time_agg_s = time.time()
for key in enc_learner_layer[0].keys():
    leaner_layer_temp = []
    for id in range(N):
        leaner_layer_temp.append(enc_learner_layer[id][key])
        #print(leaner_layer_temp)
    #print(key)
    eval_data[key] = FHE_helper.computeWeightedAverage(leaner_layer_temp, scalingFactors)
time_agg_e = time.time()
print("Secure FedAvg done.\n")

time_agg = (time_agg_e - time_agg_s)
print("Secure Agg Time: {}".format(time_agg))

#decryption
final_data = OrderedDict()

time_dec_s = time.time()
for key in learner_data_layer[0].keys():
    final_data[key] = FHE_helper.decrypt(eval_data[key], model_size[key])
time_agg_e = time.time()
print("Decryption done.\n")

time_dec = (time_dec_e - time_dec_s)
print("Decryption Time: {}".format(time_dec))